In [1]:
import sys
sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [2]:
from importlib import reload
from src.classifiers_davidjames9610.test_a.config import config_location
import importlib
import src.classifiers_davidjames9610.test_a.config as base_config
reload(base_config)
config = importlib.import_module(base_config.config_location)
reload(config)
print(config.basedir)
import time

/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands


In [3]:
# time = 20 ms
# fs = 16000 = 1/t
# windows samples = 20 * 10^-3 * 1/1600
# = 0.2 * 1600 = 800

In [4]:
import pickle
import d_classifiers_to_results
import src.misc_davidjames9610.utils as utils
reload(d_classifiers_to_results)
sls = utils.load_pickle(config.samples_labels)

In [5]:
features = utils.folder_pickles_to_dict(config.basedir + '/features', 'mfcc')

loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR40.pickle
loading mfcc_GaussNoise_SNR60.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR60.pickle
loading mfcc_GaussNoise_SNR40.pickle
loading mfcc_None_None.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR20.pickle
loading mfcc_GaussNoise_SNR10.pickle
loading mfcc_RealNoise_UNDER_WATER_DRONE_SNR10.pickle
loading mfcc_GaussNoise_SNR20.pickle


In [6]:
classifiers = utils.folder_pickles_to_dict(config.basedir + '/classifiers', 'GMMHMM')

loading GMMHMM.pickle


In [7]:
results = {}
plot_cms = True
save_cms = True

In [8]:
import src.misc_davidjames9610.utils as utils
reload(utils)
include_buffer = False
if include_buffer:
    d_classifiers_to_results.include_buffer_in_features(features, buffer_length=250, buffer_step=125)
    utils.dict_to_folder_pickles(config.basedir + '/features', features)
#     import pickle
#     pickle.dump(features, open(config.features,'wb'))

  buffering for: mfcc_GaussNoise_SNR10
  buffering for: mfcc_GaussNoise_SNR20
  buffering for: mfcc_GaussNoise_SNR40
  buffering for: mfcc_GaussNoise_SNR60
  buffering for: mfcc_None_None
  buffering for: mfcc_RealNoise_UNDER_WATER_DRONE_SNR10
  buffering for: mfcc_RealNoise_UNDER_WATER_DRONE_SNR20
  buffering for: mfcc_RealNoise_UNDER_WATER_DRONE_SNR40
  buffering for: mfcc_RealNoise_UNDER_WATER_DRONE_SNR60
Directory '/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands/features' already exists, removing old files:  False
saving / updating  mfcc_GaussNoise_SNR10
saving / updating  mfcc_GaussNoise_SNR20
saving / updating  mfcc_GaussNoise_SNR40
saving / updating  mfcc_GaussNoise_SNR60
saving / updating  mfcc_None_None
saving / updating  mfcc_RealNoise_UNDER_WATER_DRONE_SNR10
saving / updating  mfcc_RealNoise_UNDER_WATER_DRONE_SNR20
saving / updating  mfcc_RealNoise_UNDER_WATER_DRONE_SNR40
saving / updating  mfcc_RealNoise_UNDER_WATER_DRONE_SNR60


In [9]:
import d_classifiers_to_results
reload(d_classifiers_to_results)

start_time = time.time()

result_types = ['classification_annotations',
                'classification_buffer',
                'classification_annotations_valg',
                'classification_buffer_valg',
                ]
result_type = result_types[3]
results[result_type] = d_classifiers_to_results.get_classification_results(
    features, classifiers, sls, config.basedir, result_type, plot_cm=plot_cms, save_plots=save_cms)

end_time = time.time()
print(f"Execution time: {end_time - start_time:.6f} seconds")

Directory '/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands/plots/classification_buffer_valg/' already exists, removing old files:  False
type: classification_buffer_valg
For classifier: GMMHMM
already saved: ['mfcc_RealNoise_UNDER_WATER_DRONE_SNR40', 'mfcc_GaussNoise_SNR60', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR60', 'mfcc_GaussNoise_SNR40', 'mfcc_None_None', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR20', 'mfcc_GaussNoise_SNR10', 'mfcc_RealNoise_UNDER_WATER_DRONE_SNR10', 'mfcc_GaussNoise_SNR20']
Directory '/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands/results/classification_buffer_valg/GMMHMM' already exists, removing old files:  False
Execution time: 0.000319 seconds
